In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://drive.google.com/file/d/1UGrMGfb9zvbBqOvbV62G-XdUlBIAvOad/view
!wget https://drive.google.com/file/d/1jc7Dmp26km0PKRwf9u3Xcyui4SRiojcT/view

In [ ]:
#preprocessing
import pandas as pd
import os
import numpy as np

class_labels= ['N','D','C','A','F','M','O']
keyword_label_mapping  = {
    'normal':'N',
    'retinopathy':'D',
    'glaucoma':'G',
    'cataract':'C',
    'macular degeneration':'A',
    'hypertensive':'H',
    'myopia':'M',
    'lens dust':'O', 'optic disk photographically invisible':'O', 'low image quality':'O', 'image offset':'O'
}
non_decisive_labels = ["lens dust", "optic disk photographically invisible", "low image quality", "image offset"]
# if the keyword contains label outside of the above then, label them as others 'O'
def get_individual_labels(diagnostic_keywords):
    keywords = [ keyword  for keyword in diagnostic_keywords.split(',')]
    contains_normal = False
    for k in keywords:
        for label in keyword_label_mapping.keys():
            if label in k:
                if label == 'normal':
                    contains_normal = True # if found a 'normal' keyword, check if there are other keywords but keep in mind that a normal keyword was found
                else:
                    return keyword_label_mapping[label] # found a proper keyword label, use the first occurence

    # did not find a proper keyword label, see if there are labels other than non-decisive labels, if so, categorize them as 'others'
    decisive_label = False
    for k in keywords:
        if k not in non_decisive_labels and (('normal' not in k) or ('abnormal' in k)):
            decisive_label = True
    if decisive_label:
        # contains decisive label other than the normal and abnormal categories
        return 'O'
    if contains_normal:
        return 'N'
    # if any of the above criteria do not match, then return as is
    return keywords[0] # u

# write test cases
# if both left and right are normal, then the final diagnosis is also normal
def test_normal(row):
    l,r = row['Left-label'], row['Right-label']
    if l == 'N' and r == 'N' and row['N'] != 1:
        return False
    else:
        return True

def test_others(row):
    l,r = row['Left-label'], row['Right-label']
    if row['O'] == 1:
        if l == 'O' or r == 'O':
            return True
        else:
            return False
    return True



In [ ]:
import pandas as pd
import os
import numpy as np
# from preprocessing import get_individual_labels, test_normal,test_others
from glob import glob
import matplotlib.pyplot as plt


from pandas import DataFrame
from pathlib import Path
# from torchvision.transforms import img_transform,crop_nonzero
from sklearn.model_selection import train_test_split
# from transforms.transforms import
# import sys
# sys.path.append("../")
# import cv2


TRAIN_DIR="/content/drive/MyDrive"
df=pd.read_excel('/content/drive/MyDrive/ODIR-5K_Training_Annotations(Updated)_V2.xlsx')
csv_data= df.to_csv(os.path.join(TRAIN_DIR,"data.csv"))

Left_eye=df[['Left-Fundus','Left-Diagnostic Keywords']].copy()
Left_eye.columns=['Image','Labels']
# Left_eye.to_csv(os.path.join(TRAIN_DIR,'left_eye.csv'))

Right_eye=df[['Right-Fundus','Right-Diagnostic Keywords']].copy()
Right_eye.columns=['Image','Labels']
# Right_eye.to_csv(os.path.join(TRAIN_DIR,'right_eye.csv'))


keywords_left=[keyword_l for keywords_left in df['Left-Diagnostic Keywords'] for keyword_l in keywords_left.split(',')]
unique_keywords_left= set(keywords_left)
# print((unique_keywords_left))
# print(keywords_left[:10])

keywords_right=[keyword_r for keywords_right in df['Right-Diagnostic Keywords'] for keyword_r in keywords_right.split(',')]
unique_keywords_right= set(keywords_right)
# print((unique_keywords_right))
# print(keywords_right[:10])

class_labels= ['N','D','C','A','F','M','O']
keyword_label_mapping  = {
    'normal':'N',
    'retinopathy':'D',
    'glaucoma':'G',
    'cataract':'C',
    'macular degeneration':'A',
    'hypertensive':'H',
    'myopia':'M',
    'lens dust':'O', 'optic disk photographically invisible':'O', 'low image quality':'O', 'image offset':'O'
}
non_decisive_labels = ["lens dust", "optic disk photographically invisible", "low image quality", "image offset"]
# print(get_individual_labels('optic disk photographically invisible'))

df['Left-label']= df['Left-Diagnostic Keywords'].apply(get_individual_labels)
df['Right-label'] = df['Right-Diagnostic Keywords'].apply(get_individual_labels)

df[df['Left-label'].isin(non_decisive_labels)]
df[df['Right-label'].isin(non_decisive_labels)]

#for lefteye.csv

# left_data= pd.read_csv(r'data\left_eye.csv')
# left_columns = 'left_labels'
# l=[]
# for left in left_data['Labels']:
#      out_l= get_individual_labels(left)
#      l.append(out_l)


# left_data[left_columns]=l
# # print(l)
# left_data.to_csv(r'C:\Users\Dell\Desktop\grandchallenge\data\left_eye.csv',index=False)

#for righteye.csv

# right_data= pd.read_csv(r'data\right_eye.csv')
# right_columns = 'right_labels'
# r=[]
# for right in right_data['Labels']:
#      out_r= get_individual_labels(right)
#      r.append(out_r)


# right_data[right_columns]=r
# print(r)
# right_data.to_csv(r'C:\Users\Dell\Desktop\grandchallenge\data\right_eye.csv',index=False)



# find rows where both left and right have beeen processed as Normal, but the final diagnosis is not 'N
df[df.apply(test_normal, axis=1) == False]
# find rows where none of the left and right have been processed as Others, but the final diagnosis also contains 'O'
df[df.apply(test_others,axis=1) == False]

#transforms

# img_paths = glob(f'{df}/*.jpg')
# # len(img_paths)
# for i in range(len(img_paths)//10):
#     fig,ax = plt.subplots(1,2)
#     img = plt.imread(img_paths[i*10])
#     cropped_img = crop_nonzero(img)
#     ax[0].imshow(img)
#     ax[1].imshow(cropped_img)
#     plt.show()



    # create a new dataframe where each row corresponds to one image
left_fundus = df['Left-Fundus']
left_label = df['Left-label']
left_keywords = df['Left-Diagnostic Keywords']
right_fundus = df['Right-Fundus']
right_label = df['Right-label']
right_keywords = df['Right-Diagnostic Keywords']
id = df['ID']
age = df['Patient Age']
sex = df['Patient Sex']

# separate train and test split

SEED = 234
id_train, id_val = train_test_split(id,test_size=0.1,random_state=SEED)

train_left_fundus = df[df['ID'].isin(id_train)]['Left-Fundus']
train_left_label = df[df['ID'].isin(id_train)]['Left-label']
train_left_keywords = df[df['ID'].isin(id_train)]['Left-Diagnostic Keywords']

train_right_fundus = df[df['ID'].isin(id_train)]['Right-Fundus']
train_right_label = df[df['ID'].isin(id_train)]['Right-label']
train_right_keywords = df[df['ID'].isin(id_train)]['Right-Diagnostic Keywords']


val_left_fundus = df[df['ID'].isin(id_val)]['Left-Fundus']
val_left_label = df[df['ID'].isin(id_val)]['Left-label']
val_left_keywords = df[df['ID'].isin(id_val)]['Left-Diagnostic Keywords']

val_right_fundus = df[df['ID'].isin(id_val)]['Right-Fundus']
val_right_label = df[df['ID'].isin(id_val)]['Right-label']
val_right_keywords = df[df['ID'].isin(id_val)]['Right-Diagnostic Keywords']

# stack left and right columns vertically
train_fundus = pd.concat([train_left_fundus, train_right_fundus],axis=0,ignore_index=True,sort=True)
train_label = pd.concat([train_left_label,  train_right_label],axis=0,ignore_index=True,sort=True)
train_keywords = pd.concat([train_left_keywords,train_right_keywords],axis=0,ignore_index=True,sort=True)

val_fundus = pd.concat([val_left_fundus, val_right_fundus],axis=0,ignore_index=True)
val_label = pd.concat([val_left_label,val_right_label],axis=0,ignore_index=True)
val_keywords = pd.concat([val_left_keywords,val_right_keywords],axis=0,ignore_index=True)

train_df_left_right_separate_row = pd.concat([train_fundus,
                                              train_label,
                                              train_keywords],axis=1,sort=True,
                                              keys = ['fundus','label','keywords']) # stack horizontally
val_df_left_right_separate_row = pd.concat([  val_fundus,
                                              val_label,
                                              val_keywords],axis=1,sort=True,
                                              keys=['fundus','label','keywords']) # stack horizontally

cleaned_train_df = train_df_left_right_separate_row.drop(train_df_left_right_separate_row[train_df_left_right_separate_row['label'].isin(non_decisive_labels)].index)
cleaned_val_df = val_df_left_right_separate_row.drop(val_df_left_right_separate_row[val_df_left_right_separate_row['label'].isin(non_decisive_labels)].index)
cleaned_train_df.to_csv('/content/drive/MyDrive/processed_train_.csv')
cleaned_val_df.to_csv('/content/drive/MyDrive/processed_val-5K.csv')
# len(df),len(id_train),len(id_val),len(train_fundus),len(val_fundus),len(train_df_left_right_separate_row),len(val_df_left_ri



# test_root_dir = '../odir2019/ODIR-5K_Testing_Images'
# test_paths = glob(f'{test_root_dir}/*.jpg')
# test_paths = [ Path(p).name for p in test_paths]
# test_df = DataFrame(data={'fundus':test_paths})
# test_df.to_csv('../csv/processed_test_ODIR-5k.csv')










In [ ]:
#transforms
import numpy as np
import  matplotlib as plt
import torch
import torchvision.io as io
from torchvision.transforms import Lambda, Compose,ToTensor,Resize,Normalize
from torchvision.io import read_image
import torchvision.transforms.functional as tvf
from  torchvision.transforms import transforms as T
from PIL import Image

def nonzero_bounding_box(img:np.ndarray, verbose=False):
    '''
    1. split the image into four quadrants: h_left_split, h_right_split, w_top_split, w_bottom_split
    2. find the last non-zero pixel position for left and top splits
    3. find the first non-zero pixel position for right and bottom splits
    return the index of the above 4 values as bounding box (left,top,right,bottom)
    '''
    h,w,c = img.shape


    # split image into four quadrants, use the first channel
    left_half_axis_1d = img[h//2,:w//2,0].tolist()
    top_half_axis_1d = img[:h//2,w//2,0].tolist()

    right_half_axis_1d = img[h//2,w//2:,0].tolist()
    bottom_half_axis_1d = img[h//2:,w//2,0].tolist()

    # find first nonzero pixel positions, if no non-zero pixel positions exist, return lower-bounds and upper-bounds
    try:
        h_left = len(left_half_axis_1d) - left_half_axis_1d[::-1].index(0)
    except ValueError as e:
        # could not find zero in the list
        h_left = 0

    try:
        w_top = len(top_half_axis_1d) - top_half_axis_1d[::-1].index(0)
    except ValueError as e:
        w_top = 0

    try:
        h_right = w//2 + right_half_axis_1d.index(0)
    except ValueError as e:
        h_right = h

    try:
        w_bottom = h//2 + bottom_half_axis_1d.index(0)
    except ValueError as e:
        w_bottom = w

    if verbose:
        print(f'Image size {img.shape}')
        print(h_left,h_right,w_top,w_bottom)
    return h_left,h_right,w_top,w_bottom

def crop_nonzero(img, verbose=False):
    left, right, top, bottom = nonzero_bounding_box(img,verbose=verbose)
    return img[top:bottom,left:right,:]

def pad_to_largest_square(img:torch.Tensor,verbose=False):
    c,h,w = img.shape
    largest_side = max(img.shape)
    if (largest_side - h) != 0 :
        total_pad = largest_side - h
        # this is the side where we need to pad
        if total_pad % 2 == 0:
            #even padding
            top = bottom = total_pad // 2
        else:
            top = total_pad // 2
            bottom = total_pad // 2 + 1
    else:
        top = bottom = 0

    if (largest_side - w )!= 0:
        total_pad = largest_side - w
        # this is the side where we need to pad
        if total_pad % 2 == 0:
            # even padding
            left = right = total_pad // 2
        else:
            # odd padding
            left = total_pad // 2
            right = total_pad // 2 + 1
    else:
        left = right = 0

    required_pad = (left,top,right,bottom)
    padded_img =  tvf.pad(img,required_pad,fill=0,padding_mode='constant')

    if verbose:
        print('Img shape',img.shape)
        print('padding', required_pad)
    return padded_img







def read_image(img_path):
    # img_path = img.filename
    # img = io.read_image(img_path)
    # return img.copy() # return a copy to get rid of UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors.
    img = Image.open(img_path).convert("RGB")
    return img.copy()
    img_transform = Compose([
    Lambda(read_image),
    Lambda(crop_nonzero),
    ToTensor(),
    Lambda(pad_to_largest_square),
    Normalize(mean= training_img_mean,std= torch.sqrt(training_img_var))
])


def get_img_transform(img_size:int):
    base_img_transform = img_transform
    resized_img_transform = Compose([
        base_img_transform,
        Resize(size=img_size,interpolation=tvf.InterpolationMode.BILINEAR,antialias=True,)

    ])
    return resized_img_transform

def labels_to_idx(label):
    return LABELS_TO_IDX[label]

label_transform = Lambda(labels_to_idx)




training_img_var = torch.Tensor([0.0713, 0.0345, 0.0140])
training_img_mean = torch.Tensor([0.4384, 0.2866, 0.1646])

LABELS  = ['N','D','G','C','A','H','M','O']
LABELS_TO_IDX = {l:idx for idx, l in enumerate(LABELS)}

In [ ]:
#datasets
# from transforms import labels_to_idx,read_image, get_img_transform

# from preprocessing import ROOT_DIR
from torch.utils.data import Dataset
from os.path import join
import csv
# import matplotlib.pyplot as plt
from PIL import Image

# TRAIN_DIR=r"data"
def read_as_csv(csv_path):
    file_name_arr=[]
    label_arr=[]
    with open(csv_path,'r') as f:
        reader=csv.reader(f)
        next(reader)
        for row in reader:
            file_name_arr.append(row[1])
            label_arr.append(row[2])
    return(file_name_arr,label_arr)

class ImageDataset(Dataset):
    def __init__(self,csv_path,transforms:None):
        images,labels=read_as_csv(csv_path)
        self.images = images
        self.labels=labels
        self.transforms=transforms

    def __len__(self):
        return len(self.images)

    def __str__(self):
        return f"<ImageDataset with {self.__len__()} samples>"
    def __getitem__(self,index):
        # image=self.images[index]
        # label=self.labels[index]
        image_name=self.images[index]
        label_name= self.labels[index]

        image_path= join("/content/drive/MyDrive/ODIR-5K_Training_Dataset",image_name)
        # image= Image.open(image_path).convert('RGB')
        label=labels_to_idx(label_name)
        if self.transforms:
            image= self.transforms(image_path)
        return image,label


In [ ]:
!pip cache purge

Files removed: 2


In [ ]:
# Install PyTorch-Ignite
!pip install -q pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
import os

in_colab = "COLAB_TPU_ADDR" in os.environ
with_torchrun = "WORLD_SIZE" in os.environ

if in_colab:
    VERSION = !curl -s https://api.github.com/repos/pytorch/xla/releases/latest | grep -Po '"tag_name": "v\K.*?(?=")'
    !pip install --upgrade -q cloud-tpu-client==0.10 torch=={VERSION[0]} torchvision https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-{VERSION[0][:-2]}-cp38-cp38-linux_x86_64.whl

!pip list | grep torch

pytorch-ignite                   0.4.13
torch                            2.1.0+cu121
torchaudio                       2.1.0+cu121
torchdata                        0.7.0
torchsummary                     1.5.1
torchtext                        0.16.0
torchvision                      0.16.0+cu121


In [ ]:
!pip install ignite

In [ ]:
#dataloader
# from datasets import ImageDataset
# from transforms import  get_img_transform

import tensorboard
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torch
from torch import nn
from torchvision.models import resnet18
import ignite
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import ModelCheckpoint
from ignite.contrib.handlershandlers import TensorboardLogger, global_step_from_engine

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# BATCH_SIZE=16

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.model = resnet50(num_classes=8)

        self.model.conv1 = self.model.conv1 = nn.Conv2d(
            3, 64, kernel_size=3, padding=1, bias=False
        )

    def forward(self, x):
        return self.model(x)


model = Net().to(device)


train_loader = DataLoader(
    ImageDataset(csv_path=r'C:\Users\Dell\Desktop\grandchallenge\data\processed_train_.csv' ,transforms= get_img_transform(img_size=(224, 224))),batch_size=5,shuffle=True)


val_loader = DataLoader(
    ImageDataset(csv_path=r'C:\Users\Dell\Desktop\grandchallenge\data\processed_val-5K.csv',transforms=get_img_transform(img_size=(224, 224))),batch_size=5,shuffle=False
)

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

trainer = create_supervised_trainer(model, optimizer, criterion, device)

val_metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(criterion)
}

train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)




log_interval = 100

@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")


def score_function(engine):
    return engine.state.metrics["accuracy"]




model_checkpoint = ModelCheckpoint(
    "checkpoint",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer),
    require_empty= False
)

val_evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})

tb_logger = TensorboardLogger(log_dir="tb-logger")

tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batch_loss": loss},
)

for tag, evaluator in [("training", train_evaluator), ("validation", val_evaluator)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer),
    )

trainer.run(train_loader, max_epochs=5)

tb_logger.close()




ModuleNotFoundError: No module named 'ignite'

In [ ]:
!pip search pytorch-ignite

ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.
